In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
import os
import csv
import pandas as pd
import wfdb  # Make sure wfdb is installed

output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

# N = normal 
# S = supra-ventricular premature
# V = ventricular escape
# F = fusion of ventricular and normal
# Q = unclassified heartbeats
symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}

for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample

        # Filter out symbols not in symbol_to_category
        filtered_symbols_annotations = [(sym, ann) for sym, ann in zip(symbols, annotations) if sym in symbol_to_category]
        categories = [symbol_to_category[sym] for sym, ann in filtered_symbols_annotations]
        annotations_filtered = [ann for sym, ann in filtered_symbols_annotations]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations_filtered})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("Done")

Done


In [3]:
import os
import pandas as pd
import numpy as np


def process_patient_data(patient_number, data_creation_dir="data_creation"):
    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]
    second_column_name = ecg_df.columns[1] if len(ecg_df.columns) > 1 else None

    sampling_rate = 360  # Hz
    total_window_size_seconds = 1  # Total window size in seconds
    total_window_size_samples = total_window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        # Randomly determine the window split around the annotation point
        before_seconds = np.random.uniform(0, total_window_size_seconds)
        after_seconds = total_window_size_seconds - before_seconds
        before_samples = int(before_seconds * sampling_rate)
        after_samples = int(after_seconds * sampling_rate)
        
        start_point = max(0, annotation_point - before_samples)
        end_point = start_point + total_window_size_samples  # Ensure the window is of the exact expected size
        
        # Adjust the end point if it exceeds the length of the data
        if end_point > len(ecg_df):
            end_point = len(ecg_df)
            start_point = max(0, end_point - total_window_size_samples)  # Adjust start point accordingly
        
        # Process data from the first column
        window_data_first_column = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        
        patient_X.append(window_data_first_column)
        patient_Y.append(category)
        
        # If there's a second column, process it and add as a new entry
        if second_column_name:
            window_data_second_column = ecg_df.iloc[start_point:end_point][second_column_name].to_numpy()
            
            patient_X.append(window_data_second_column)
            patient_Y.append(category)  # Repeat category for the new entry
    
    return patient_X, patient_Y



# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[-0.145, -0.145, -0.145, ..., -0.445, -0.475, -0.51 ],
       [-0.065, -0.065, -0.065, ..., -0.36 , -0.355, -0.305],
       [-0.295, -0.285, -0.285, ..., -0.245, -0.25 , -0.275],
       ...,
       [ 0.035,  0.03 ,  0.04 , ..., -0.025, -0.015, -0.005],
       [-0.24 , -0.245, -0.235, ..., -0.305, -0.33 , -0.365],
       [ 0.07 ,  0.07 ,  0.065, ...,  0.05 ,  0.025,  0.   ]])

In [5]:
Y

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [6]:
df_x = pd.DataFrame(X)
df_x

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,-0.325,-0.330,-0.350,-0.365,-0.360,-0.380,-0.425,-0.445,-0.475,-0.510
1,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.080,-0.080,...,-0.235,-0.250,-0.270,-0.280,-0.290,-0.300,-0.330,-0.360,-0.355,-0.305
2,-0.295,-0.285,-0.285,-0.290,-0.295,-0.310,-0.290,-0.295,-0.300,-0.305,...,-0.225,-0.215,-0.240,-0.245,-0.240,-0.245,-0.235,-0.245,-0.250,-0.275
3,-0.205,-0.185,-0.195,-0.195,-0.220,-0.220,-0.210,-0.215,-0.220,-0.220,...,-0.190,-0.190,-0.200,-0.185,-0.165,-0.145,-0.145,-0.170,-0.175,-0.170
4,-0.330,-0.335,-0.330,-0.320,-0.345,-0.355,-0.340,-0.330,-0.325,-0.330,...,-0.405,-0.420,-0.420,-0.435,-0.420,-0.410,-0.410,-0.405,-0.425,-0.430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,0.110,0.100,0.095,0.100,0.100,0.090,0.085,0.090,0.110,0.115,...,-0.010,-0.035,-0.045,-0.060,-0.055,-0.045,-0.030,-0.025,-0.010,0.005
218984,-0.365,-0.355,-0.350,-0.335,-0.285,-0.210,-0.080,0.120,0.335,0.620,...,-0.185,-0.195,-0.190,-0.200,-0.190,-0.195,-0.185,-0.185,-0.175,-0.160
218985,0.035,0.030,0.040,0.045,0.040,0.040,0.020,0.010,0.020,0.050,...,-0.100,-0.100,-0.090,-0.075,-0.075,-0.055,-0.045,-0.025,-0.015,-0.005
218986,-0.240,-0.245,-0.235,-0.245,-0.250,-0.240,-0.220,-0.210,-0.225,-0.225,...,-0.325,-0.370,-0.360,-0.315,-0.275,-0.280,-0.285,-0.305,-0.330,-0.365


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_x = scaler.fit_transform(df_x)
df_x = pd.DataFrame(df_x)

In [8]:
import pywt

def madev(d, axis=None):
    'Mean absolute deviation of a signal'
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def wavelet_denoising(x, wavelet='sym4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


df_x = wavelet_denoising(df_x, wavelet='sym4', level=2)
df_x = pd.DataFrame(df_x)

In [9]:
df_y = pd.DataFrame(Y, columns=['ColumnName'])
print(df_y)

       ColumnName
0               N
1               N
2               N
3               N
4               N
...           ...
218983          N
218984          N
218985          N
218986          N
218987          N

[218988 rows x 1 columns]


In [10]:
value_counts_y = df_y['ColumnName'].value_counts()
print(value_counts_y)

N    181262
Q     16086
V     14472
S      5562
F      1606
Name: ColumnName, dtype: int64


In [11]:
df_fusionné = pd.concat([df_y, df_x], axis=1)
df_fusionné

,ColumnName,0,1,2,3,4,5,6,7,8,...,350,351,352,353,354,355,356,357,358,359
0,N,0.462657,0.482548,0.484076,0.480509,0.483015,0.486312,0.486521,0.485864,0.486532,...,0.465025,0.465029,0.464824,0.464402,0.465701,0.466904,0.465382,0.464323,0.457096,0.448836
1,N,0.482615,0.482878,0.483020,0.482990,0.483850,0.485008,0.486124,0.487286,0.488439,...,0.481455,0.483832,0.481466,0.477827,0.472715,0.467783,0.468938,0.472220,0.476545,0.481245
2,N,0.475144,0.474839,0.474512,0.474153,0.473762,0.473363,0.472934,0.472513,0.472204,...,0.474959,0.475280,0.475449,0.475538,0.475561,0.475534,0.475536,0.475511,0.475464,0.475375
3,N,0.482240,0.482096,0.481937,0.481760,0.481573,0.481380,0.481172,0.480965,0.480794,...,0.482115,0.482213,0.482278,0.482323,0.482342,0.482344,0.482360,0.482364,0.482364,0.482342
4,N,0.459802,0.460632,0.461160,0.461892,0.463549,0.465220,0.467821,0.470120,0.468647,...,0.464370,0.466179,0.466420,0.465949,0.465477,0.464773,0.463647,0.462465,0.461005,0.459663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,N,0.501811,0.502516,0.503368,0.504333,0.505167,0.506002,0.506880,0.507790,0.508841,...,0.501709,0.502507,0.502670,0.502566,0.502473,0.502314,0.502085,0.501890,0.501591,0.501384
218984,N,0.473994,0.469905,0.466114,0.464172,0.471147,0.483583,0.494215,0.507023,0.533669,...,0.480714,0.480821,0.481596,0.482347,0.483304,0.484080,0.482570,0.479992,0.479009,0.478194
218985,N,0.497407,0.498265,0.500164,0.502181,0.501787,0.501135,0.500174,0.499762,0.504447,...,0.492979,0.493363,0.493525,0.493654,0.493771,0.493935,0.494445,0.495133,0.495757,0.496500
218986,N,0.468849,0.470084,0.471408,0.472946,0.474360,0.475672,0.477183,0.478569,0.479031,...,0.478180,0.466354,0.463121,0.463822,0.468308,0.474283,0.474005,0.471990,0.470205,0.468458


In [12]:
df_fusionné.to_csv('df_fusionné.csv', index=False)